# Advance Data Types

We will cover few advance data types in this section such as 
- Collections
- set
- frozenset

## Collections 

The collections module contains many specialized container datatypes as shown below

| Name | Description|
|:-------------:|---------------|
|namedtuple() |	factory function for creating tuple subclasses with named fields|
|deque 	|list-like container with fast appends and pops on either end|
|ChainMap 	|dict-like class for creating a single view of multiple mappings|
|Counter |	dict subclass for counting hashable objects|
|OrderedDict |	dict subclass that remembers the order entries were added|
|defaultdict |	dict subclass that calls a factory function to supply missing values|
|UserDict |	wrapper around dictionary objects for easier dict subclassing|
|UserList 	|wrapper around list objects for easier list subclassing|
|UserString |	wrapper around string objects for easier string subclassing|

### ChainMap — Search Multiple Dictionaries

The `ChainMap` class manages a list of dictionaries, and can be used to search through them in the order they are added to find values for associated keys. 

It makes a good **"context" container**, as it can be visualised as a stack for which changes happen as soon as the stack grows, with these changes being discarded again as soon as the stack shrinks.

Treat it as a view table in DB, where actual values are still stored in their respective table and we can still perform all the operation on them.

#### Accessing Values

The ChainMap supports the same API as a regular dictionary for accessing existing values.

In [1]:
import collections
# from collections import ChainMap

a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}

m = collections.ChainMap(a, b)

print('Individual Values')
print('a = {}'.format(m['a']))
print('b = {}'.format(m['b']))
print('c = {}'.format(m['c']))
print("-"*20)

print(type(m.keys()))
print('Keys = {}'.format(list(m.keys())))
print('Values = {}'.format(list(m.values())))
print("-"*20)

print('Items:')
for k, v in m.items():
    print('{} = {}'.format(k, v))
print("-"*20)

print('"d" in m: {}'.format(('d' in m)))

Individual Values
a = A
b = B
c = C
--------------------
<class 'collections.abc.KeysView'>
Keys = ['b', 'c', 'a']
Values = ['B', 'C', 'A']
--------------------
Items:
b = B
c = C
a = A
--------------------
"d" in m: False


In [2]:
a = {'a': 'A', 'c': 'C'}
b = {'b': 'B', 'c': 'D'}

m = collections.ChainMap(a, b)

lst = []

for v in m.keys():
    lst.append(v)

for v in m.values():
    lst.append(v)

print(lst)

['b', 'c', 'a', 'B', 'C', 'A']


The child mappings are searched in the order they are passed to the constructor, so the value reported for the key 'c' comes from the a dictionary.

#### Reordering

The ChainMap stores the list of mappings over which it searches in a list in its maps attribute. This list is mutable, so it is possible to add new mappings directly or to change the order of the elements to control lookup and update behavior.

In [3]:
import collections

a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}

cm = collections.ChainMap(a, b)

print(cm.maps)
print('c = {}\n'.format(cm['c']))
# reverse the list
cm.maps = list(reversed(cm.maps)) # m = collections.ChainMap(b, a)

print(cm.maps)
print('c = {}'.format(cm['c']))

[{'a': '1', 'c': '3'}, {'b': '2', 'c': '33'}]
c = 3

[{'b': '2', 'c': '33'}, {'a': '1', 'c': '3'}]
c = 33


When the list of mappings is reversed, the value associated with 'c' changes.

#### Updating Values

A ChainMap does not cache the values in the child mappings. Thus, if their contents are modified, the results are reflected when the ChainMap is accessed.

In [4]:
import collections

a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}

m = collections.ChainMap(a, b)
print('Before: {}'.format(m['c']))
a['c'] = '3.3'
print('After : {}'.format(m['c']))

Before: 3
After : 3.3


In [5]:
import collections

a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}

cm = collections.ChainMap(b, a)
print(cm.maps)
print('Before: {}'.format(cm['c']))
a['c'] = '3.3'
print('After : {}'.format(cm['c']))

[{'b': '2', 'c': '33'}, {'a': '1', 'c': '3'}]
Before: 33
After : 33


Changing the values associated with existing keys and adding new elements works the same way.

It is also possible to set values through the ChainMap directly, although only the first mapping in the chain is actually modified.

In [6]:
import collections

a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}

cm = collections.ChainMap(a, b)
print('Before: {}'.format(cm['c']))
cm['c'] = '3.3'
print('After : {}'.format(cm['c']))
print(a['c'])
print(b['c'])

Before: 3
After : 3.3
3.3
33


In [7]:
import collections

a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}

cm = collections.ChainMap(b, a)
print('Before: {}'.format(cm['c']))
cm['c'] = '3.3'
print('After : {}'.format(cm['c']))
print(a['c'])
print(b['c'])

Before: 33
After : 3.3
3
3.3


In [8]:
import collections

a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}

cm = collections.ChainMap(a, b)
print('Before: {}'.format(cm['c']))
cm['d'] = '3.3'
print('After : {}'.format(cm['c']))
print(cm.maps)
print(a)
print(b)

Before: 3
After : 3
[{'a': '1', 'c': '3', 'd': '3.3'}, {'b': '2', 'c': '33'}]
{'a': '1', 'c': '3', 'd': '3.3'}
{'b': '2', 'c': '33'}


When the new value is stored using m, the a mapping is updated.

ChainMap provides a convenience method for creating a new instance with one extra mapping at the front of the maps list to make it easy to avoid modifying the existing underlying data structures.

This stacking behavior is what makes it convenient to use ChainMap instances as template or application contexts. Specifically, it is easy to add or update values in one iteration, then discard the changes for the next iteration.

In [9]:
import collections


a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}

m1 = collections.ChainMap(a, b)
m2 = m1.new_child()

print('m1 before:', m1)
print('m2 before:', m2)

m2['c'] = '3.3'

print('m1 after:', m1)
print('m2 after:', m2)

m1 before: ChainMap({'a': '1', 'c': '3'}, {'b': '2', 'c': '33'})
m2 before: ChainMap({}, {'a': '1', 'c': '3'}, {'b': '2', 'c': '33'})
m1 after: ChainMap({'a': '1', 'c': '3'}, {'b': '2', 'c': '33'})
m2 after: ChainMap({'c': '3.3'}, {'a': '1', 'c': '3'}, {'b': '2', 'c': '33'})


For situations where the new context is known or built in advance, it is also possible to pass a mapping to new_child().

In [10]:
import collections


a = {'a': '1', 'c': '3'}
b = {'b': '2', 'c': '33'}
c = {'c': '333'}

m1 = collections.ChainMap(a, b)
m2 = m1.new_child(c)

print('m1["c"] = {}'.format(m1['c']))
print('m2["c"] = {}'.format(m2['c']))
print(m2)

#This is the equivalent of
m2_1 = collections.ChainMap(c, *m1.maps)
print(m2_1)

m1["c"] = 3
m2["c"] = 333
ChainMap({'c': '333'}, {'a': '1', 'c': '3'}, {'b': '2', 'c': '33'})
ChainMap({'c': '333'}, {'a': '1', 'c': '3'}, {'b': '2', 'c': '33'})


### Counter

*Counter* is a *dict* subclass which helps count the hashable objects. It stores elements as dictionary keys and the counts of the objects as value. In other words , It is a container that keeps track of how many times equivalent values are present.

**For example:**

In [11]:
# Tally occurrences of words in a list
from collections import Counter

cnt = Counter()
for word in ['red', 'blue', 'red', 'green', 'blue', 'blue']:
    cnt[word] += 1

Counter({'blue': 3, 'red': 2, 'green': 1})

# Find the ten most common words in Hamlet
import re
words = re.findall(r'\w+', open('hamlet.txt').read().lower())
Counter(words).most_common(10)

[('the', 1150),
 ('and', 983),
 ('to', 772),
 ('of', 672),
 ('i', 638),
 ('you', 556),
 ('a', 550),
 ('my', 516),
 ('in', 450),
 ('it', 419)]

Where as Counter can be used:

#### `Counter()` with lists

In [12]:
l = [1 ,23 , 23, 44, 4, 44, 55, 555, 44, 32, 23, 44, 56, 64, 2, 1]

lstCounter = Counter(l)
print(lstCounter)
print(lstCounter.most_common(4))

Counter({44: 4, 23: 3, 1: 2, 4: 1, 55: 1, 555: 1, 32: 1, 56: 1, 64: 1, 2: 1})
[(44, 4), (23, 3), (1, 2), (4, 1)]


#### `Counter` with Strings

In [13]:
sentance = "The collections module is a tresure trove of a built-in module that implements " + \
           "specialized container datatypes providing alternatives to Python’s general purpose " + \
           "built-in containers."

wordList = sentance.split(" ")
Counter(wordList).most_common(3)

[('module', 2), ('a', 2), ('built-in', 2)]

#### `Counter` methods

In [14]:
# find the most common words

# Methods with Counter()
c = Counter(wordList)
print(c.most_common(4))
print(c.items())

[('module', 2), ('a', 2), ('built-in', 2), ('The', 1)]
dict_items([('The', 1), ('collections', 1), ('module', 2), ('is', 1), ('a', 2), ('tresure', 1), ('trove', 1), ('of', 1), ('built-in', 2), ('that', 1), ('implements', 1), ('specialized', 1), ('container', 1), ('datatypes', 1), ('providing', 1), ('alternatives', 1), ('to', 1), ('Python’s', 1), ('general', 1), ('purpose', 1), ('containers.', 1)])


### Default dict 

The standard dictionary includes the method `setdefault()` for retrieving a value and establishing a default if the value does not exist. By contrast, defaultdict lets the caller specify the default up front when the container is initialized.

In [15]:
d = {"a": 1, "b": 2}
print(d)
print(d['a'])
try:
    print(d['d'])
except Exception as e:
    print("Error:", e)

{'a': 1, 'b': 2}
1
Error: 'd'


In [16]:
print(d.get('d', None))

None


In [17]:
from collections import defaultdict

dd  = defaultdict(object)
print(dd)
print(dd['one'])
print(dd)
dd['Two'] = 2
print(dd)
for d in dd:
    print(d)
    print(dd[d])

defaultdict(<class 'object'>, {})
defaultdict(<class 'object'>, {'one': <object object at 0x10f369290>})
defaultdict(<class 'object'>, {'one': <object object at 0x10f369290>, 'Two': 2})
one
Two
2


In [18]:
from collections import defaultdict

def default_func():
    return 'defaul value'

d1 = defaultdict(default_func)
print(d1)

defaultdict(<function default_func at 0x10f468a70>, {})


In [19]:
d1['test']

'defaul value'

In [20]:
print(d1.items())

dict_items([('test', 'defaul value')])


In [21]:
print(d1.values(), " : ",  d1.keys())

dict_values(['defaul value'])  :  dict_keys(['test'])


In [22]:
# Using factory function
import collections

def default_factory():
    return 'default value'

d = collections.defaultdict(default_factory, india='New Delhi')
print('d:', d)
print('india =>', d['india'])
print('India =>', d['India'])
print('bar =>', d['bar'])
print(d)

d: defaultdict(<function default_factory at 0x10f4685f0>, {'india': 'New Delhi'})
india => New Delhi
India => default value
bar => default value
defaultdict(<function default_factory at 0x10f4685f0>, {'india': 'New Delhi', 'India': 'default value', 'bar': 'default value'})


In [23]:
# Using factory function
import collections

def default_val():
    return 'Bhopal'

d = collections.defaultdict(default_factory, 
                            {
                                "India": 'New Delhi', 
                                 "Karnataka":"Bangaluru"
                            })

In [24]:
print('d:', d)
print('india =>', d['india'])
print('MP =>', d['MP'])
print(d)

d: defaultdict(<function default_factory at 0x10f4685f0>, {'India': 'New Delhi', 'Karnataka': 'Bangaluru'})
india => default value
MP => default value
defaultdict(<function default_factory at 0x10f4685f0>, {'India': 'New Delhi', 'Karnataka': 'Bangaluru', 'india': 'default value', 'MP': 'default value'})


In [25]:
# Using factory function

# ---------------------------------------------------
# TODO:  How can i pass value to the default function
# ---------------------------------------------------
import collections

def default_factory():
    return 'default value'

d = collections.defaultdict(default_factory, foo='bar')
print('d:', d)
print('foo =>', d['foo'])
print('bar =>', d['bar'])

d: defaultdict(<function default_factory at 0x10f4688c0>, {'foo': 'bar'})
foo => bar
bar => default value


In [26]:
# Using list as the default_factory, it is easy to group a sequence of key-value pairs into a dictionary of lists:

from collections import defaultdict

countryList = [("India", "New Delhi"), ("Iceland", "Reykjavik"), 
               ("Indonesia", "Jakarta"), ("Ireland", "Dublin"), 
               ("Israel", "Jerusalem"), ("Italy", "Rome")]
d = defaultdict(list)
for country, capital in countryList:
    d[country].append(capital)

print(d.items())

dict_items([('India', ['New Delhi']), ('Iceland', ['Reykjavik']), ('Indonesia', ['Jakarta']), ('Ireland', ['Dublin']), ('Israel', ['Jerusalem']), ('Italy', ['Rome'])])


In [27]:
# Setting the default_factory to int makes the defaultdict useful for counting
quote = 'Vande Mataram'
dd = defaultdict(int)
print(dd)
for chars in quote:
    dd[chars] += 1
    
print(dd.items())
print(dd['T'])

defaultdict(<class 'int'>, {})
dict_items([('V', 1), ('a', 4), ('n', 1), ('d', 1), ('e', 1), (' ', 1), ('M', 1), ('t', 1), ('r', 1), ('m', 1)])
0


### deque — Double-Ended Queue

A double-ended queue, or deque, supports adding and removing elements from either end of the queue. The more commonly used stacks and queues are degenerate forms of deques, where the inputs and outputs are restricted to a single end.

In [28]:
import collections

d = collections.deque('Vande Mataram')
print('Deque:', d)
print('Length:', len(d))
print('Left end:', d[0])
print('Right end:', d[-1])

d.remove('e')
print('remove(e):', d)

Deque: deque(['V', 'a', 'n', 'd', 'e', ' ', 'M', 'a', 't', 'a', 'r', 'a', 'm'])
Length: 13
Left end: V
Right end: m
remove(e): deque(['V', 'a', 'n', 'd', ' ', 'M', 'a', 't', 'a', 'r', 'a', 'm'])


#### Adding 

In [29]:
import collections

# Add to the right
d1 = collections.deque()
d1.extend('Vande')
print('extend    :', d1)

for a in " Mataram":
    d1.append(a)
    
d1.extend(" !!!")  
print('append    :', d1)
d1.extendleft(" #!* ")  
print('append    :', d1)

# Add to the left
d2 = collections.deque()
d2.extendleft(range(6))
print('extendleft:', d2)
d2.appendleft(6)
print('appendleft:', d2)

extend    : deque(['V', 'a', 'n', 'd', 'e'])
append    : deque(['V', 'a', 'n', 'd', 'e', ' ', 'M', 'a', 't', 'a', 'r', 'a', 'm', ' ', '!', '!', '!'])
append    : deque([' ', '*', '!', '#', ' ', 'V', 'a', 'n', 'd', 'e', ' ', 'M', 'a', 't', 'a', 'r', 'a', 'm', ' ', '!', '!', '!'])
extendleft: deque([5, 4, 3, 2, 1, 0])
appendleft: deque([6, 5, 4, 3, 2, 1, 0])


#### Consuming

Python provides two methods (`pop` & `popleft`) which can be used to consume deque. We will encounter `IndexError` exception when calling `pop` or `popleft` if nothing to pop. We are using it to exit out of our while loop.

In [30]:
d1 = collections.deque("~~~~ P O K ....")

while True:
    try:
        print(d1.pop(), end="")
    except IndexError:
        break
print("")
print(d1)
d1 = collections.deque("!!! .... Vande Mataram .... !!!")
print(d1)
while True:
    try:
        print(d1.popleft(), end=" ")
    except IndexError:
        break
print("")
print(d1)

.... K O P ~~~~
deque([])
deque(['!', '!', '!', ' ', '.', '.', '.', '.', ' ', 'V', 'a', 'n', 'd', 'e', ' ', 'M', 'a', 't', 'a', 'r', 'a', 'm', ' ', '.', '.', '.', '.', ' ', '!', '!', '!'])
! ! !   . . . .   V a n d e   M a t a r a m   . . . .   ! ! ! 
deque([])


#### Rotating

`deque` provides `rotate` method to rotate the queue as shown in the below example. We can change the direction of rotation by changing the rotation number from `+ve` to `-ve`.
- `-ve` number will rotate the queue left side 
- `+ve` number will rotate the queue right side 


In [31]:
d = collections.deque("Philippines")

print(d)
d.rotate(4)
print(d)
d.rotate(-4)
print(d)

deque(['P', 'h', 'i', 'l', 'i', 'p', 'p', 'i', 'n', 'e', 's'])
deque(['i', 'n', 'e', 's', 'P', 'h', 'i', 'l', 'i', 'p', 'p'])
deque(['P', 'h', 'i', 'l', 'i', 'p', 'p', 'i', 'n', 'e', 's'])


We can obtain string from deque using `"".join` similar to how we get it in `list`

In [32]:
print("".join(d))

Philippines


#### `deque` vs `list` performance comparison

In [33]:
d = collections.deque(range(999999))

In [34]:
%%timeit
while True:
    try:
        d1.pop()
    except IndexError:
        break

500 ns ± 26.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [35]:
l = list(range(999999))

In [36]:
%%timeit
while True:
    try:
        l.pop()
    except IndexError:
        break

540 ns ± 33.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [37]:
print(l)

[]


### OrderedDict

It is a dictionary subclass that remembers the order in which its contents are added.

Lets start with a normal dictionary:

In [38]:
fruitsCount = {}
fruitsCount["apple"] = 10
fruitsCount["grapes"] = 120
fruitsCount["mango"] = 200
fruitsCount["kiwi"] = 2000
fruitsCount["leeche"] = 20
print(fruitsCount)
for fruit in fruitsCount:
    print(fruit)

{'apple': 10, 'grapes': 120, 'mango': 200, 'kiwi': 2000, 'leeche': 20}
apple
grapes
mango
kiwi
leeche


In [39]:
# Now lets try this with OrderedDict

from collections import OrderedDict as OD

fruitsCount = OD()
fruitsCount["apple"] = 10
fruitsCount["grapes"] = 120
fruitsCount["mango"] = 200
fruitsCount["kiwi"] = 2000
fruitsCount["leeche"] = 20
print(fruitsCount)
for fruit in fruitsCount:
    print(fruit)

OrderedDict([('apple', 10), ('grapes', 120), ('mango', 200), ('kiwi', 2000), ('leeche', 20)])
apple
grapes
mango
kiwi
leeche


### namedtuple

Named tuples helps to have meaning of each position in a tuple and allow us to code with better readability and self-documenting code. You can use them in any place where you are using tuples. In the example we will create a namedtuple to show hold information for points.

In [2]:
from collections import namedtuple

States = namedtuple("India", ['MP', 'K', "AP"])  # Defining the namedtuple

In [3]:
vals = States("Bhopal", K='Bangaluru', AP = 'Hyderabad')  # Creating an object
print(vals)
print(vals.MP , vals.K , vals.AP)
print(vals[0] , vals[1])  # Accessing the values in normal way

India(MP='Bhopal', K='Bangaluru', AP='Hyderabad')
Bhopal Bangaluru Hyderabad
Bhopal Bangaluru


### References

* https://docs.python.org/3/library/collections.html, 
* http://alexmarandon.com/articles/python_collections_tips/, 
* http://pymbook.readthedocs.io/en/latest/collections.html

## Practice Questions

* Write a function `lensort` to sort a list of strings based on length.
* Write a program to count frequency of characters in a given file. Can you use character frequency to tell whether the given file is a Python program file, C program file or a text file?
* Write a program similar to 'tail'
* write a program similar to "wc"